In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
my_drive = '/content/drive/My Drive/'

In [5]:
!pip install transformers

In [6]:
!pip install -U sentence-transformers

Processing /root/.cache/pip/wheels/12/a5/1c/03b7d87e027121fe1e23048007594e73f39a23e833658529c7/sentence_transformers-0.2.4.1-cp36-none-any.whl


In [0]:
#!cp '/content/drive/My Drive/roberta_finetuned_stsb.zip' roberta_finetuned_stsb.zip

In [0]:
#!unzip roberta_finetuned_stsb.zip

In [9]:
"""
This examples trains BERT for the STSbenchmark from scratch. It generates sentence embeddings
that can be compared using cosine-similarity to measure the similarity.
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime


#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [10]:
# Read the dataset
train_batch_size = 16
num_epochs = 4

model_save_path = 'output/training_stsb_roberta-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#sts_reader = STSDataReader('stsbenchmark', normalize_scores=True)
# parameters for STSDataReader -> dataset_folder, s1_col_idx=5, s2_col_idx=6, score_col_idx=4, delimiter="\t", quoting=csv.QUOTE_NONE, normalize_scores=True, min_score=0, max_score=5
sts_reader = STSDataReader('', s1_col_idx=0, s2_col_idx=1, score_col_idx=2, normalize_scores=True)

# Use RoBERTa for mapping tokens to embeddings
word_embedding_model = models.RoBERTa('roberta-base')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=True)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

2019-12-13 01:47:54 - https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json not found in cache or force_download set to True, downloading to /tmp/tmpnn2ziw5x


100%|██████████| 473/473 [00:00<00:00, 282728.49B/s]


2019-12-13 01:47:55 - copying /tmp/tmpnn2ziw5x to cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.9dad9043216064080cf9dd3711c53c0f11fe2b09313eaa66931057b4bdcaf068
2019-12-13 01:47:55 - creating metadata file for /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.9dad9043216064080cf9dd3711c53c0f11fe2b09313eaa66931057b4bdcaf068
2019-12-13 01:47:55 - removing temp file /tmp/tmpnn2ziw5x
2019-12-13 01:47:55 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.9dad9043216064080cf9dd3711c53c0f11fe2b09313eaa66931057b4bdcaf068
2019-12-13 01:47:55 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,


100%|██████████| 501200538/501200538 [00:16<00:00, 30296141.89B/s]


2019-12-13 01:48:12 - copying /tmp/tmp1eui0dt6 to cache at /root/.cache/torch/transformers/228756ed15b6d200d7cb45aaef08c087e2706f54cb912863d2efe07c89584eb7.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
2019-12-13 01:48:14 - creating metadata file for /root/.cache/torch/transformers/228756ed15b6d200d7cb45aaef08c087e2706f54cb912863d2efe07c89584eb7.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
2019-12-13 01:48:14 - removing temp file /tmp/tmp1eui0dt6
2019-12-13 01:48:14 - loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-pytorch_model.bin from cache at /root/.cache/torch/transformers/228756ed15b6d200d7cb45aaef08c087e2706f54cb912863d2efe07c89584eb7.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
2019-12-13 01:48:18 - https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json not found in cache or force_download set to True, downloading to /tmp/tmpmuze0e_y


100%|██████████| 898823/898823 [00:00<00:00, 2102407.89B/s]


2019-12-13 01:48:19 - copying /tmp/tmpmuze0e_y to cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2019-12-13 01:48:19 - creating metadata file for /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2019-12-13 01:48:19 - removing temp file /tmp/tmpmuze0e_y
2019-12-13 01:48:19 - https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt not found in cache or force_download set to True, downloading to /tmp/tmpd92f1rhq


100%|██████████| 456318/456318 [00:00<00:00, 1337302.34B/s]


2019-12-13 01:48:20 - copying /tmp/tmpd92f1rhq to cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
2019-12-13 01:48:20 - creating metadata file for /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
2019-12-13 01:48:20 - removing temp file /tmp/tmpd92f1rhq
2019-12-13 01:48:20 - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2019-12-13 01:48:20 - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.7

In [15]:
sts_reader = STSDataReader('', s1_col_idx=0, s2_col_idx=1, score_col_idx=2, normalize_scores=True)

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")
train_data = SentencesDataset(sts_reader.get_examples(my_drive+'clinicalSTS2019.train_train60.csv'), model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples(my_drive+'clinicalSTS2019.train_validate20.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)


Convert dataset:  14%|█▍        | 137/992 [00:00<00:00, 1366.55it/s]

2019-12-13 01:50:07 - Read STSbenchmark train dataset


Convert dataset: 100%|██████████| 992/992 [00:00<00:00, 1973.38it/s]


2019-12-13 01:50:07 - Num sentences: 992
2019-12-13 01:50:07 - Sentences 0 longer than max_seqence_length: 0
2019-12-13 01:50:07 - Sentences 1 longer than max_seqence_length: 0
2019-12-13 01:50:07 - Read STSbenchmark dev dataset


Convert dataset: 100%|██████████| 331/331 [00:00<00:00, 2330.79it/s]

2019-12-13 01:50:08 - Num sentences: 331
2019-12-13 01:50:08 - Sentences 0 longer than max_seqence_length: 0
2019-12-13 01:50:08 - Sentences 1 longer than max_seqence_length: 0


In [16]:
# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_data)*num_epochs/train_batch_size*0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


2019-12-13 01:50:11 - Warmup-steps: 25


In [17]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Iteration:  37%|███▋      | 23/62 [00:10<00:19,  2.00it/s]

KeyboardInterrupt: ignored

In [0]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_data = SentencesDataset(examples=sts_reader.get_examples("clinicalSTS2019.train_test20.csv"), model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(test_dataloader)
model.evaluate(evaluator)

In [0]:
!zip -r output.zip output

In [0]:
!cp output.zip '/content/drive/My Drive/roberta_finetuned_sts_biomedical.zip'